In [2]:
sc

''

In [2]:
textRDD = sc.newAPIHadoopFile('data/Moby-Dick.txt',
                              'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
                              'org.apache.hadoop.io.LongWritable',
                              'org.apache.hadoop.io.Text',
                              conf={'textinputformat.record.delimiter': "\r\n\r\n"}) \
.map(lambda x: x[1])

sentences=textRDD.map(lambda x:x.replace('\r\n',' '))\
        .flatMap(lambda x: x.split(". "))

In [3]:
# Function for cleaning punctuation
def cleanData(inputstr):
    s=list(inputstr) # convert string to list of characters
    for i in range(len(s)):
        o=ord(s[i])             # ASCII value of each string
        if o==46 or (o>=48 and o<=57) or (o>=97 and o<=122) : #space, numbers and lowercase left as is
            continue 
        elif (o>=65 and o<=90): # convert uppercase to lowercase
            s[i]=chr(o+32)  
        else:               # replace with space for anything else 
            s[i]=" " 
        
    return str("".join(s)) # str to convert unicode string to string

In [4]:
sentences_rm_punct=sentences.map(cleanData)#sentences_rm_rn.map(cleanData) # clean data of punctuation, uppercase

In [5]:
sentence2words=sentences_rm_punct.map(lambda x: x.split(" "))\
    .filter(lambda x: x != ['']) # split by space and remove blank sentences

In [6]:
def removeBlankWords(inputlist):
    y=[]
    for x in inputlist:
        if x != '':
            y.append(x)
    return y
sentence2words_clean=sentence2words.map(removeBlankWords) # remove blank words of form ''

In [7]:
def printOutput(n,freq_ngramRDD):
    top=freq_ngramRDD.take(5)
    print '\n============ %d most frequent %d-grams'%(5,n)
    print '\nindex\tcount\tngram'
    for i in range(5):
        print '%d.\t%d: \t"%s"'%(i+1,top[i][0],' '.join(top[i][1]))

In [8]:
for n in range(1,6):
    if n==1:
        pairs=sentence2words_clean.flatMap(lambda x:x)
        freq_ngramRDD=pairs.map(lambda word: (word, 1))\
             .reduceByKey(lambda a, b: a + b)\
             .sortBy(lambda x:x[1],False)\
             .map(lambda x: (x[1],[x[0]]))
    else:
        if n==2:
            pairs=sentence2words_clean.flatMap(lambda x:zip(x,x[1:])) # create bigram
        elif n==3:
            pairs=sentence2words_clean.flatMap(lambda x:zip(x,x[1:],x[2:])) # create trigram
        elif n==4:
            pairs=sentence2words_clean.flatMap(lambda x:zip(x,x[1:],x[2:],x[3:])) #create 4-grams
        elif n==5:
            pairs=sentence2words_clean.flatMap(lambda x:zip(x,x[1:],x[2:],x[3:],x[4:])) #create 5-grams

        freq_ngramRDD=pairs.map(lambda word: (word, 1)) \
                 .reduceByKey(lambda a, b: a + b)\
                 .sortBy(lambda x:x[1],False)\
                 .map(lambda x: (x[1],x[0]))
        
    printOutput(n,freq_ngramRDD)


============ 5 most frequent 1-grams

index	count	ngram
1.	14620: 	"the"
2.	6731: 	"of"
3.	6502: 	"and"
4.	4785: 	"a"
5.	4702: 	"to"

============ 5 most frequent 2-grams

index	count	ngram
1.	1906: 	"of the"
2.	1193: 	"in the"
3.	746: 	"to the"
4.	444: 	"from the"
5.	404: 	"the whale"

============ 5 most frequent 3-grams

index	count	ngram
1.	114: 	"the sperm whale"
2.	102: 	"of the whale"
3.	88: 	"the white whale"
4.	64: 	"one of the"
5.	58: 	"out of the"

============ 5 most frequent 4-grams

index	count	ngram
1.	42: 	"of the sperm whale"
2.	27: 	"the sperm whale s"
3.	20: 	"at the same time"
4.	18: 	"project gutenberg tm electronic"
5.	18: 	"of the whale s"

============ 5 most frequent 5-grams

index	count	ngram
1.	13: 	"the project gutenberg literary archive"
2.	12: 	"project gutenberg literary archive foundation"
3.	11: 	"project gutenberg tm electronic works"
4.	11: 	"of the sperm whale s"
5.	10: 	"and at the same time"
